In [1]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import copy

# Created: Hanjie Liu (j13 '24) Fall 2023
# Last Edited: Fall 2023 (j13 '24)

C:\Users\jiefi\AppData\Local\Temp\ipykernel_5896\989104526.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Defaults
Files needed (in files directory):
1. Show order as a txt file. Can see `fall-2023-show-order.txt` as example (the content is just copied off show order spreadsheet however).

In [13]:
# EDIT THIS
client_ssh_name = 'client_secret.json'
spreadsheed_name = '[Fall 2023] Master Roster'
spreadsheet_file = 'files\[Fall 2023] Master Roster - Roster.csv'
show_order_path = 'files/fall-2023-show-order.txt'

# Read files

Get list of dances in show order

In [3]:
# puts the dance names into show order

dance_names = []
# Open the text file and read its content
with open(show_order_path, 'r') as file:
    for line in file:
        # Split each line into columns based on tabs
        columns = line.strip().split('\t')

        # Check if the line has at least two columns (index and dance name)
        if len(columns) >= 2:
            # Extract the dance name from the second column and append to the list
            dance_name = columns[1]
            dance_names.append(dance_name)

# Print the list of dance names
print(dance_names)
print(len(dance_names)) # should be 18
dance_names += ['Choreog', 'Exec'] # add choreog and exec

['Super', 'Dreams of Dai', '90s Love', 'Youth', 'The Eve', 'AYAYAYA', 'A Rainy Dawn', 'Secret', 'Spirit of Ink', 'PINKCHELLA', 'Big Fish', 'VENI VIDI VICI', 'Autumn Ripples', 'Butterfly', 'Broken Melodies', 'The Call of the Past', 'Teddy Bear', 'Blue']
18


Get master roster as a dataframe

In [14]:
df = pd.read_csv(spreadsheet_file)
df

,First Name,Nickname,Last Name,Email Address,Year,Program Name,# Dances,Super,Dreams of Dai,90s Love,...,VENI VIDI VICI,Autumn Ripples,Butterfly,Broken Melodies,The Call of the Past,Teddy Bear,Blue,Choreog,Exec,Lions
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,11,13,...,23,13,21,32,10,33,29,22,11,4
1,Abena,NaN,Kyereme-Tuah,abenakt@mit.edu,Sophomore,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abigail,NaN,Xu,agxu@mit.edu,Freshman,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN
3,Adam,NaN,Zweiger,adamz@mit.edu,Freshman,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN
4,Addison,NaN,Spiegel,spiegel1@mit.edu,Freshman,NaN,2.0,NaN,NaN,x,...,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,Yunbeen,NaN,Bae,yunbeen@mit.edu,Senior,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN
243,Yuru,NaN,Lin,yurulin@mit.edu,Freshman,NaN,2.0,NaN,NaN,drop,...,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,Yutong,NaN,Zhang,zyt0318@mit.edu,Graduate,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,Yuxin,Candy,Xie,yxie25@mit.edu,Junior,Candy Xie,2.0,NaN,late drop,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(client_ssh_name, scope)
client = gspread.authorize(creds)

spreadsheet = client.open(spreadsheed_name)
roster_worksheet = spreadsheet.get_worksheet(0)

df = pd.DataFrame(roster_worksheet.get_all_records())
print('columns match # of dances?', set(df.columns[7:-1]) == set(dance_names))
df

columns match # of dances? True


,Name,Nickname,Last Name,Email Address,Year,Program Name,# Dances,Super,Dreams of Dai,90s Love,...,VENI VIDI VICI,Autumn Ripples,Butterfly,Broken Melodies,The Call of the Past,Teddy Bear,Blue,Choreog,Exec,Lions
0,,,,,,,,34,11,11,...,23,13,21,32,10,33,29,22,11,4
1,Abena,,Kyereme-Tuah,abenakt@mit.edu,Sophomore,,1,,,,...,,,,,,,,,,
2,Abigail,,Xu,agxu@mit.edu,Freshman,,2,,,,...,,,,x,,,,,,
3,Adam,,Zweiger,adamz@mit.edu,Freshman,,1,,,,...,,,,x,,,,,,
4,Addison,,Spiegel,spiegel1@mit.edu,Freshman,,2,,,x,...,,,,x,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,Yunbeen,,Bae,yunbeen@mit.edu,Senior,,1,,,,...,,,,,,x,,,,
243,Yuru,,Lin,yurulin@mit.edu,Freshman,,2,,,drop,...,x,,,,,,,,,
244,Yutong,,Zhang,zyt0318@mit.edu,Graduate,,1,,,,...,,,,,,,,,,
245,Yuxin,Candy,Xie,yxie25@mit.edu,Junior,Candy Xie,2,,late drop,,...,,,,,,,,,,


# Generate Program Names

Generates a list from the master roster of dances and dancer names in show order (dances) + alphabetical order (names). Uses program names and adds nicknames.

In [5]:
# helper function. prints lists as a pretty string.
def print_list(List):
    string_list = string_names = str(List).replace("'", "").replace("[", "").replace("]", "")
    return string_list

Outputs a string, and then you can go make a doc for choreographers to review / double check. Expand to get the full string.

In [32]:
for my_dance in dance_names:
    #my_dance = "90s Love"
    my_dancers = []
    for index, row in df.iterrows():
        # looks thru each dancer
        if row[my_dance] == "x":
            if not row['Program Name']:
                name = row['Name']
                if row['Nickname'] and row['Nickname'] != row['Name']:
                    name += " (" + row['Nickname'] + ")"
                name += " " + row['Last Name']
            else:
                name = row['Program Name']
            
            my_dancers.append(name)
    print(my_dance)
    print(print_list(my_dancers))
    print("")

Super
Alicia Lin, Amber Wang, Angela Gao, Angela Jiao, Annie Lin, Arthur Liang, Ashley Ke, Brandon Chen, Bryan Wong, Chris Zhang, Chris Schmidt-Hong, Diego Coello, Eva Goldie, Aron Qiu, Gabriela Erin (Erin) Mariangel, Jack Jin, Jeffery Li, Josleen St. Luce, Kingston Lew, Lana Xu, Matt Feng, Melody Yu, Michael Zhang, Michelle Wu, Mingzhen (Cynthia) Qi, Muhua (Cici) Xu, Nicholas (Nick) Tsao, Nicole Xu, Ningshan (Karen) Ma, Olivia Lee, Robin Liu, Robin Xiong, Vivian Han, Yu (Angela) Shi

Dreams of Dai
Amanda Mei, Amy Chang, Amy Zhong, Cecilia Huang, Dora Zhou, Elizabeth Ke, Hanjie Liu, Peggy Yang, Sravani Duggirala, Jayden Wu, Yuetong Xu

90s Love
Addison Spiegel, Adithya Balachandran, Alexis Yi, Alicia Lin, Annie Feng, Emily Huang, Izar Vargas, Kevin Kurashima, Maria Cortez, Maxim (Maxi) Attiogbe, William Lin

Youth
Angela Li, Derek Yen, Dora Hu, Jia Yi (Jessica) Zhang, Kevin Qian, Rachel Loh, Sophie Liu, Jayden Wu

The Eve
Alyssa Solomon, Amber Wang, Angela Zhang, Angelina Zheng, Austin